In [1]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW
from tqdm import tqdm
from joblib import dump, load

In [2]:
dic = load("new_data_512.joblib")

In [19]:
len(str(dic['probability'][1]).split())

109542

In [ ]:
data_time=dic

In [6]:
sample_data=[]
for i in range(0,5):
    sample_data.append(dic['time series'][i])

In [15]:
sample_data=dic['time series'].copy()

In [16]:
len(sample_data)

133

In [25]:
sample_data.clear()
data_before.clear()
dic.clear()

In [22]:
data_before=dic.copy()

In [17]:
class TextDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=512):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

        if tokenizer.pad_token_id is None:
            tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        inputs = self.tokenizer(text, return_tensors="pt", max_length=self.max_length, truncation=True, padding="max_length")
        
        # Get input IDs and attention mask
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']

        # Ensure pad_token_id is set
        if 'pad_token_id' not in inputs:
            inputs['pad_token_id'] = self.tokenizer.eos_token_id
        
        return input_ids, attention_mask

# Function to fine-tune the model
def fine_tune_model(model, tokenizer, dataset, batch_size=4, num_epochs=1, learning_rate=5e-5):
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    optimizer = AdamW(model.parameters(), lr=learning_rate)

    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for input_ids, attention_mask in tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            optimizer.zero_grad()
            input_ids = input_ids.squeeze(dim=1).to(device)
            attention_mask = attention_mask.squeeze(dim=1).to(device)
            labels = input_ids.clone()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(dataloader)}")

# Function to interactively test the model
def interactively_test_model(model, tokenizer):
    while True:
        user_input = input("Enter text (or 'exit' to quit): ")
        if user_input.lower() == 'exit':
            break
        input_ids = tokenizer.encode(user_input, return_tensors="pt").to(device)
        output = model.generate(input_ids, max_length=100, num_return_sequences=1)
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        print("Generated text:", generated_text)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained model and tokenizer
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)

# Example dataset (replace this with your own data loading logic)


# Fine-tune the model
dataset = TextDataset(sample_data, tokenizer)
fine_tune_model(model, tokenizer, dataset, batch_size=2, num_epochs=1)

# Interactively test the model

Epoch 1/1: 100%|███████████████████████████████████████████████████████████████████████| 67/67 [12:04<00:00, 10.82s/it]

Epoch 1/1, Loss: 2.922529515935414


In [18]:
interactively_test_model(model, tokenizer)

Enter text (or 'exit' to quit): time series is


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated text: time series is not a good idea, and we can't be sure.

We can't.

We can't be sure.

We can't be sure.

We can't be sure.

We can't be sure.

We can't be sure.

We can't be sure.

We can't be sure.


We can't be sure.

We can't be sure.
We can't be sure
Enter text (or 'exit' to quit): exit
